# Consultando clientes

Las consultas son las funciones más sencillas, tenemos dos:

* Listar todos los clientes
* Buscar cliente a partir del DNI

## Listar todos los clientes

Para listar los clientes lo haremos en la ruta `/clientes/` de la API, podemos probar algo así:

```python
import database as db

@app.get("/clientes/")
async def clientes():
    content = db.Clientes.lista
    headers = {"content-type": "charset=utf-8"}
    return JSONResponse(content=content, headers=headers)
```

Aunque tiene cierto sentido no funcionará, nos dará este error al ejecutar la petición http://127.0.0.1:8000/clientes/:

```bash
TypeError: Object of type Cliente is not JSON serializable
```

Como bien indica necesitamos que la clase `Cliente` sea serializable, lo cuál en este contexto significa que debe tener estructura de diccionario para que `JSONResponse` la pueda procesar correctamente.

Podemos hacerlo al vuelo con comprensión de listas, no es muy elegante pero como esto es un ejercicio de aprendizaje nos servirá:

```python
content = [
    {'dni': cliente.dni, 'nombre': cliente.nombre, 'apellido': cliente.apellido}
    for cliente in db.Clientes.lista
]
```

Este será el resultado:

```json
[{"dni":"15J","nombre":"Marta","apellido":"Pérez"},
 {"dni":"48H","nombre":"Manolo","apellido":"López"},
 {"dni":"28Z","nombre":"Mariana","apellido":"García"}]
```

En este punto es recomendable crear un método to_dict en nuestra clase `Cliente`, eso nos permitirá ahorrarnos transformar constantemente los clientes a mano:

```python
class Cliente:
    def __init__(self, dni, nombre, apellido):
        self.dni = dni
        self.nombre = nombre
        self.apellido = apellido

    def __str__(self):
        return f"({self.dni}) {self.nombre} {self.apellido}"

    def to_dict(self):
        return {'dni': self.dni, 'nombre': self.nombre, 'apellido': self.apellido}
```

Y simplemente lo utilizaremos:

```python
content = [cliente.to_dict() for cliente in db.Clientes.lista]
```


## Buscar cliente a partir del DNI

En esta petición necesitamos enviar un campo dinámico (el dni). `FastAPI` nos facilita la tarea permitiéndonos programar estos campos como variables en la definición. 

Necesitamos una ruta, por ejemplo `/clientes/buscar/{dni}/` de tipo `GET`:

```python
@app.get("/clientes/buscar/{dni}/")
async def clientes_buscar(dni: str):
    cliente = db.Clientes.buscar(dni=dni)
    headers = {"content-type": "charset=utf-8"}
    return JSONResponse(content=cliente.to_dict(), headers=headers)
```

Si realizamos la petición pasando un DNI que exista http://127.0.0.1:8000/cliente/15J/ nos devolverá el cliente:

```json
{"dni":"15J","nombre":"Marta","apellido":"Pérez"}
```

En caso de no existir tendremos un error interno:

```json
Internal Server Error
```

El error lo veremos en la terminal:

```bash
AttributeError: 'NoneType' object has no attribute 'dni'
```

Debemos tratar este caso y devolver un error `404` en lugar de realizar la serialización de algo nulo:

```python
from fastapi import FastAPI, Response, HTTPException

@app.get("/clientes/buscar/{dni}/")
async def clientes_buscar(dni: str):
    cliente = db.Clientes.buscar(dni=dni)
    if not cliente:
        raise HTTPException(status_code=404, detail="Cliente no encontrado")
    headers = {"content-type": "charset=utf-8"}
    return JSONResponse(content=cliente.to_dict(), headers=headers)
```

Ahora al buscar un cliente inexistente aparecerá un error con información:

```json
{"detail":"Cliente no encontrado"}
```

Con esto tenemos las consultas listas.